## 2.1: Building the model

In [9]:
using JuMP
using Gurobi
using Random

###2.1 FINAL LIGE FORELØBIG

#------
# DATA
#------

K = 3 # number vehicles
N = 6 # number of nodes, node 1 is the depot
T = 8 # number of periods, which is days

D_k = 900 # capacity of truck k
C_i = [0 502 488 220 486 742] #storage capacity
h_c = 0.025 #holding cost
init_inventory = [0 98.8 153.9 42.4 23.4 85.7]

demand_mean = [0 0 0 0 0 0 0 0 
75.9 75.9	75.9	75.9	75.9	85.7	85.7	75.9
62.9	94.8	94.8	94.8	94.8	68.65	36.75	62.9
67.2	67.2	67.2	67.2	67.2	67.15	67.15	67.2
102.3	156	156	156	156	119.05	65.35	102.3
107.4	130.1	130.1	130.1	130.1	65.15	42.45	107.4]

c = [0	140	434	389	419	125
140	0	300	455	400	97
434	300	0	609	417	330
389	455	609	0	256	358
419	400	417	256	0	316
125	97	330	358	316	0]

holding_cost = 0.025

#------
# MODEL
#------

model = Model(Gurobi.Optimizer)

@variable(model, x[1:N, 1:N, 1:K, 1:T] >= 0, Bin) #If there is delivered from i to j
@variable(model, y[1:N, 1:K, 1:T] >= 0, Bin) 
@variable(model, z[1:N, 1:K, 1:T] >= 0) # load of truck k when arriving at node i
@variable(model, q[1:N, 1:K, 1:T] >= 0) # Quantity salmon delivered at node i, in truck k, in period t
@variable(model, I[1:N, 0:T] >= 0) # Inventory level at node i in period t

@objective(
    model,
    Min,
    sum(sum(0.025 * I[i, t] for i = 2:N) for t = 1:T) + sum(
        sum(
            sum(sum(c[i, j] * x[i, j, k, t] for j = 1:N) for i = 1:N) for
            k = 1:K
        ) for t = 1:T
    )
)



# VRP constraints formulations:
#@constraint(model,[i = 2:N, t = 1:T], sum(y[i,k,t] for k in 1:K) == 1);
##### Inflow = outflow in depot. 
@constraint(model, [h = 1:N, k = 1:K, t = 1:T], sum(i == h ? 0 : x[i, h, k, t] for i = 1:N) == y[h, k, t]);
@constraint(model, [h = 1:N, k = 1:K, t = 1:T], sum(j == h ? 0 : x[h, j, k, t] for j = 1:N) == y[h, k, t]);

#### Demand for DC cannot exceed quantity delivered from trucks
#@constraint(model, [k = 1:K, t = 1:T], sum(demand_mean[i, t] * y[i, k, t] for i = 2:N) <= D_k)

####load of truck constraint VED IKKE HELT HVAD DENNE GØR
@constraint(model,[i = 2:N, j = 2:N, k = 1:K, t = 1:T], z[i, k, t] - demand_mean[i, t] >= z[j, k, t] - (1 - x[i, j, k, t]) * sum(demand_mean[i, t] for i = 2:N))


#INVENTORY CONSTRAINTS FORMULATION#
####Inflow = outflow in depot
@constraint(model, [t = 2:T], I[1, t-1] == I[1, t] + sum(sum(q[i, k, t] for i = 2:N) for k = 1:K))

####Inventory level at node in in period 0 is equal to initial inventory level
@constraint(model, [i = 1:N], I[i, 0] == init_inventory[i])

####inflow day-1 = outflow in DC's today. 
@constraint(model, [i = 2:N, t = 1:T], I[i, t-1] + sum(q[i, k, t] for k = 1:K) == I[i, t] + demand_mean[i, t])

####Constraints on storage on DC's capacity
@constraint(model, [i = 2:N, t = 1:T], I[i, t-1] + sum(q[i, k, t] for k = 1:K) <= C_i[i])

###CONNECTING VRP AND IRP
@constraint(model, [i = 2:N, k = 1:K, t = 1:T], q[i, k, t] <= C_i[i] * y[i, k, t])
@constraint(model,[k = 1:K, t = 1:T], sum(q[i, k, t] for i = 2:N) <= D_k * y[1, k, t])

#-------
# SOLVE
#-------

optimize!(model)

println()
for t = 1:T
    print("Period ", t, "\n")
    for k = 1:K
        has_route = false
        for i = 1:N
            for j = 1:N
                if (value(x[i, j, k, t]) == 1)
                    print(i, "->", j, " ")
                    has_route = true
                end
            end
        end
        if has_route
            println()
        end
    end
end
println()
for t = 1:T
    for k = 1:K
        for i = 1:N
            if (value(q[i, k, t]) > 0)
                println(
                    "\nq[",
                    i,
                    ",",
                    k,
                    ",",
                    t,
                    "] = ",
                    value(q[i, k, t]),
                )
            end
        end
    end
end
#    println();
for i = 1:N
    println("\nI(", i, ",t)= ", JuMP.value.(I[i, :]))
end

print("\ntotal cost = ")
println(objective_value(model))


print("\ntotal cost = ")
println(objective_value(model))


Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-05-14
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1125 rows, 1350 columns and 4157 nonzeros
Model fingerprint: 0xf09e89cd
Variable types: 342 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [3e-02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 7e+02]
Presolve removed 168 rows and 213 columns
Presolve time: 0.01s
Presolved: 957 rows, 1137 columns, 3764 nonzeros
Variable types: 273 continuous, 864 integer (864 binary)

Root relaxation: objective 2.649403e+03, 705 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | 

## 2.2 Build a simulation

In [10]:

std = [0 0 0 0 0 0 0 0 
11.87	11.87	11.87	11.87	11.87	14.69	14.69	11.87
10.91	13.89	13.89	13.89	13.89	14.14	11.22	10.91
11.27	11.27	11.27	11.27	11.27	12.86	12.86	11.27
13.19	16.03	16.03	16.03	16.03	15.05	11.98	13.19
13.04	14.00	14.00	14.00	14.00	11.08	9.84	13.04] #calculated from data

#initalize matrix with zeros
fullfilled = zeros(Int, (N, T));
Random.seed!();
iterations = 10000
filled= zeros((N, T));
sales = zeros((N, T));
for r = 1:iterations
    actual_demand = demand_mean + randn(N,T).*std;
    new_inv = value.(I)
    for t=1:T #T
        for i =1:N #N
            delivered = 0;
            for k = 1:K
                delivered = delivered + JuMP.value.(q[i,k,t]);
            end
            # update inventory and calculate fill rate:
            if t==1
                new_inv[i,t] = init_inventory[i] + delivered - actual_demand[i,t];
                sales[i,t] = min(new_inv[i,t] + delivered, actual_demand[i,t]);
            else
                new_inv[i,t] = new_inv[i,t-1] + delivered - actual_demand[i,t];
                sales[i,t] = min(new_inv[i,t-1]+ delivered, actual_demand[i,t]);
            end
            if value(sales[i,t]) >= actual_demand[i,t] * 0.95
                filled[i,t] += 1;
            end
        end
    end
end
fullfill_rate = filled/iterations
println(fullfill_rate)
#println(fullfill_rate[6,8]) #sanity check


[1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0; 1.0 1.0 1.0 0.9996 0.5567 1.0 0.9925 0.5359; 1.0 1.0 1.0 0.9998 0.5694 0.9992 0.9683 0.5333; 1.0 0.6898 1.0 0.9988 0.5516 1.0 0.9886 0.5408; 1.0 0.6564 1.0 1.0 0.6332 1.0 0.9968 0.5591; 1.0 1.0 1.0 1.0 1.0 1.0 0.9987 0.5585]


## 2.3 Safety factor to meet fill target rate

In [11]:
sf=1.93
demand_mean_new = demand_mean + std*sf;

#NEW SIMULATION WITH UNCERTAINTY
fullfilled = zeros(Int, (N, T));
Random.seed!();
iterations = 10000
filled= zeros((N, T));
sales = zeros((N, T));
for r = 1:iterations
    actual_demand = demand_mean_new + randn(N,T).*std;
    new_inv = zeros(N, T+1);
    for t=1:T #T
        for i =1:N #N
            delivered = 0;
            for k = 1:K
                delivered = delivered + JuMP.value.(q[i,k,t]);
            end
            # update inventory and calculate fill rate:
            if t==1
                new_inv[i,t] = init_inventory[i] + delivered - actual_demand[i,t];
                sales[i,t] = min(new_inv[i,t] + delivered, actual_demand[i,t]);
            else
                new_inv[i,t] = new_inv[i,t-1] + delivered - actual_demand[i,t];
                sales[i,t] = min(new_inv[i,t-1]+ delivered, actual_demand[i,t]);
            end
            if value(sales[i,t]) >= actual_demand[i,t] * 0.95
                filled[i,t] += 1;
            end
        end
    end
end

fullfill_rate = filled/iterations
println(fullfill_rate)
println(fullfill_rate[6,8])

count = 0
for i=1:N
    for t=1:T
        if fullfill_rate[i,t] > 0.95
            count += 1
        end
    end
end
percentage = count/(N*T)
println(percentage)

[1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0; 1.0 1.0 1.0 0.326 0.0 0.7971 0.0027 0.0; 1.0 1.0 1.0 0.4906 0.0 0.0613 0.0008 0.0; 0.9966 0.0123 1.0 0.2523 0.0 0.5781 0.0036 0.0; 1.0 0.0113 1.0 0.9531 0.0002 0.4719 0.0099 0.0; 1.0 1.0 1.0 1.0 0.9986 0.4941 0.0335 0.0]
0.0
0.5


## 3.1 Capacity utilization of solution from part 2

In [14]:
utility_storage = zeros(N,T);
for i = 1:N
    for t = 1:T
        #utility_storage[i,t] = value.(I[i,t])/C_i[i];
        utility_storage[i,t] = value.(I[i,t-1]+sum(q[i, k, t] for k=1:K))/C_i[i];
    end
end
print("utility storage matrix:")
utility_storage

utility storage matrix:

6×8 Matrix{Float64}:
 NaN         Inf        Inf        …  Inf        NaN         NaN
   0.755976   0.604781   0.453586      0.492629    0.321912    0.151195
   0.905943   0.777049   0.582787      0.344877    0.204201    0.128893
   0.63       0.324545   0.916364      0.915909    0.610682    0.305455
   0.531481   0.320988   0.970782      0.589918    0.344959    0.210494
   0.824798   0.680054   0.815768  …   0.289757    0.201954    0.144744

In [13]:
trucks_utility = zeros(K,T);
for k = 1:K
    for t = 1:T
        trucks_utility[k,t] = sum(value.(q[i,k,t]) for i=1:N)/D_k;
    end
end
print("truck storage matrix:")
trucks_utility

truck storage matrix:

3×8 Matrix{Float64}:
 0.0       0.0  1.0  0.0  0.0  0.0  0.0  0.0
 1.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.584778  0.0  0.0  0.0  0.0  1.0  0.0  0.0

## 3.2 scenarios

In [16]:
K = 3 # number vehicles
N = 6 # number of nodes, node 1 is the depot
T = 8 # number of periods, which is days

D_k = 900*10 # capacity of truck k #inflate only with 10 for scenario a
C_i = [0 502 488 220 486 742]*10 #storage capacity # inflate only with 10 for scenario b
h_c = 0.025 #holding cost
init_inventory = [0 98.8 153.9 42.4 23.4 85.7]

demand_mean = [0 0 0 0 0 0 0 0 
75.9 75.9	75.9	75.9	75.9	85.7	85.7	75.9
62.9	94.8	94.8	94.8	94.8	68.65	36.75	62.9
67.2	67.2	67.2	67.2	67.2	67.15	67.15	67.2
102.3	156	156	156	156	119.05	65.35	102.3
107.4	130.1	130.1	130.1	130.1	65.15	42.45	107.4]
demand_old = demand_mean

#sf=0
#sf=1 #for scenario a
#sf=1.39 #for scenario b
sf=0.567 #scenario c

demand_mean = demand_mean + std*sf;

c = [0	140	434	389	419	125
140	0	300	455	400	97
434	300	0	609	417	330
389	455	609	0	256	358
419	400	417	256	0	316
125	97	330	358	316	0]

holding_cost = 0.025

#------
# MODEL
#------

model = Model(Gurobi.Optimizer)

@variable(model, x[1:N, 1:N, 1:K, 1:T] >= 0, Bin) #If there is delivered from i to j
@variable(model, y[1:N, 1:K, 1:T] >= 0, Bin) 
@variable(model, z[1:N, 1:K, 1:T] >= 0) # load of truck k when arriving at node i
@variable(model, q[1:N, 1:K, 1:T] >= 0) # Quantity salmon delivered at node i, in truck k, in period t
@variable(model, I[1:N, 0:T] >= 0) # Inventory level at node i in period t

@objective(
    model,
    Min,
    sum(sum(0.025 * I[i, t] for i = 2:N) for t = 1:T) + sum(
        sum(
            sum(sum(c[i, j] * x[i, j, k, t] for j = 1:N) for i = 1:N) for
            k = 1:K
        ) for t = 1:T
    )
)



# VRP constraints formulations:
#@constraint(model,[i = 2:N, t = 1:T], sum(y[i,k,t] for k in 1:K) == 1);
##### Inflow = outflow in depot. 
@constraint(model, [h = 1:N, k = 1:K, t = 1:T], sum(i == h ? 0 : x[i, h, k, t] for i = 1:N) == y[h, k, t]);
@constraint(model, [h = 1:N, k = 1:K, t = 1:T], sum(j == h ? 0 : x[h, j, k, t] for j = 1:N) == y[h, k, t]);

#### Demand for DC cannot exceed quantity delivered from trucks
#@constraint(model, [k = 1:K, t = 1:T], sum(demand_mean[i, t] * y[i, k, t] for i = 2:N) <= D_k)

####load of truck constraint VED IKKE HELT HVAD DENNE GØR
@constraint(model,[i = 2:N, j = 2:N, k = 1:K, t = 1:T], z[i, k, t] - demand_mean[i, t] >= z[j, k, t] - (1 - x[i, j, k, t]) * sum(demand_mean[i, t] for i = 2:N))


#INVENTORY CONSTRAINTS FORMULATION#
####Inflow = outflow in depot
@constraint(model, [t = 2:T], I[1, t-1] == I[1, t] + sum(sum(q[i, k, t] for i = 2:N) for k = 1:K))

####Inventory level at node in in period 0 is equal to initial inventory level
@constraint(model, [i = 1:N], I[i, 0] == init_inventory[i])

####inflow day-1 = outflow in DC's today. 
@constraint(model, [i = 2:N, t = 1:T], I[i, t-1] + sum(q[i, k, t] for k = 1:K) == I[i, t] + demand_mean[i, t])

####Constraints on storage on DC's capacity
@constraint(model, [i = 2:N, t = 1:T], I[i, t-1] + sum(q[i, k, t] for k = 1:K) <= C_i[i])

###CONNECTING VRP AND IRP
@constraint(model, [i = 2:N, k = 1:K, t = 1:T], q[i, k, t] <= C_i[i] * y[i, k, t])
@constraint(model,[k = 1:K, t = 1:T], sum(q[i, k, t] for i = 2:N) <= D_k * y[1, k, t])

#-------
# SOLVE
#-------

optimize!(model)

println()
for t = 1:T
    print("Period ", t, "\n")
    for k = 1:K
        has_route = false
        for i = 1:N
            for j = 1:N
                if (value(x[i, j, k, t]) == 1)
                    print(i, "->", j, " ")
                    has_route = true
                end
            end
        end
        if has_route
            println()
        end
    end
end
println()
for t = 1:T
    for k = 1:K
        for i = 1:N
            if (value(q[i, k, t]) > 0.01)
                println(
                    "\nq[",
                    i,
                    ",",
                    k,
                    ",",
                    t,
                    "] = ",
                    value(q[i, k, t]),
                )
            end
        end
    end
end
#    println();
for i = 1:N
    println("\nI(", i, ",t)= ", JuMP.value.(I[i, :]))
end


print("\ntotal cost = ")
println(objective_value(model))


####2.2
"""set random seed"""

std = [0 0 0 0 0 0 0 0 
11.87	11.87	11.87	11.87	11.87	14.69	14.69	11.87
10.91	13.89	13.89	13.89	13.89	14.14	11.22	10.91
11.27	11.27	11.27	11.27	11.27	12.86	12.86	11.27
13.19	16.03	16.03	16.03	16.03	15.05	11.98	13.19
13.04	14.00	14.00	14.00	14.00	11.08	9.84	13.04]

#initalize matrix with zeros
fullfilled = zeros(Int, (N, T));
Random.seed!(0);
iterations = 10000
filled= zeros((N, T));
sales = zeros((N, T));
for r = 1:iterations
    actual_demand = demand_old + randn(N,T).*std;
    new_inv = zeros(N, T+1);
    for t=1:T #T
        for i =1:N #N
            delivered = 0;
            for k = 1:K
                delivered = delivered + JuMP.value.(q[i,k,t]);
            end
            # update inventory and calculate fill rate:
            if t==1
                new_inv[i,t] = init_inventory[i] + delivered - actual_demand[i,t];
                sales[i,t] = min(new_inv[i,t] + delivered, actual_demand[i,t]);
            else
                new_inv[i,t] = new_inv[i,t-1] + delivered - actual_demand[i,t];
                sales[i,t] = min(new_inv[i,t-1]+ delivered, actual_demand[i,t]);
            end
            if value(sales[i,t]) >= actual_demand[i,t] * 0.95
                filled[i,t] += 1;
            end
        end
    end
end

fullfill_rate = filled/iterations
println(fullfill_rate)
println(fullfill_rate[6,8])

count = 0
for i=1:N
    for t=1:T
        if fullfill_rate[i,t] > 0.95
            count += 1
        end
    end
end
percentage = count/(N*T)
println(percentage)

utility_storage = zeros(N,T);
for i = 1:N
    for t = 1:T
        #utility_storage[i,t] = value.(I[i,t])/C_i[i];
        utility_storage[i,t] = value.(I[i,t-1]+sum(q[i, k, t] for k=1:K))/C_i[i];
    end
end
print("utility storage matrix")
utility_storage
"""
trucks_utility = zeros(K,T);
for k = 1:K
    for t = 1:T
        trucks_utility[k,t] = sum(value.(q[i,k,t]) for i=1:N)/D_k;
    end
end
print("truck storage matrix")
trucks_utility 
"""

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-05-14
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1125 rows, 1350 columns and 4157 nonzeros
Model fingerprint: 0xf3126ba7
Variable types: 342 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+03]
  Objective range  [3e-02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 7e+03]
Found heuristic solution: objective 9244.4215308
Presolve removed 168 rows and 213 columns
Presolve time: 0.01s
Presolved: 957 rows, 1137 columns, 3764 nonzeros
Variable types: 273 continuous, 864 integer (864 binary)

Root relaxation: objective 3.045282e+02, 666 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds    

"trucks_utility = zeros(K,T);\nfor k = 1:K\n    for t = 1:T\n        trucks_utility[k,t] = sum(value.(q[i,k,t]) for i=1:N)/D_k;\n    end\nend\nprint(\"truck storage matrix\")\ntrucks_utility \n"